
# **Reading the sample bookstore dataset.**

In [ ]:
import pandas as pd

# Assume books_upd dataframe is already created as per the provided script
books_upd = pd.read_excel(r"C:\Users\hp\Desktop\MMA-McGill\INSY-660\booksummaries.xls")

books_upd['genre'] = books_upd.apply(lambda row: [value for value in [row['Genre 1'], row['Genre 2'], row['Genre 3'], row['Genre 4'], row['Genre 5'], row['Genre 6'], row['Genre 7'], row['Genre 8'], row['Genre 9']] if pd.notnull(value)], axis=1)

books_info = {} 
for num in range(len(books_upd)):
    try:
        books_info[books_upd["Title"][num]] = {
            "author": books_upd["Author"][num], 
            "genre": books_upd["genre"][num], 
            "length": books_upd["length"][num],  
            "price": books_upd["price"][num],
            "bestseller": books_upd["bestseller"][num],  # New column
            "age_group": books_upd["age_group"][num]  # New column
        }
    except KeyError:
        continue


# **Defining our Recommendation bot**

In [ ]:

class BookBuddyBot:
    def __init__(self):
        self.user_data = {}
        self.categories = {
            "fiction": [
                "Absurdist fiction", "Adventure novel", "Alternate history", "Apocalyptic and post-apocalyptic fiction",
                "Children's literature", "Comedy", "Crime Fiction", "Cyberpunk", "Detective fiction", "Drama",
                "Erotica", "Existentialism", "Fantasy", "Historical fiction", "Horror", "Humour", "Mystery",
                "Novel", "Novella", "Parody", "Play", "Poetry", "Psychological novel", "Roman", "Romance novel",
                "Satire", "Science Fiction", "Spy fiction", "Supernatural", "Suspense", "Thriller",
                "Utopian and dystopian fiction", "Vampire fiction", "War novel"
            ],
            "non-fiction": [
                "Historical novel", "History", "Autobiographical novel", "Anthropology", "Autobiography",
                "Business", "Computer Science", "Economics", "Inspirational", "Memoir", "Philosophy", "Religion",
                "Science", "Self-help", "Sociology", "Spirituality", "Time travel"
            ]
        }
        self.length_options = [
            "Short (less than 200 pages)", 
            "Medium (200-400 pages)", 
            "Long (400-600 pages)", 
            "Very Long (more than 600 pages)"
        ]
        self.price_options = [
            "5-10", 
            "10-20", 
            "20-30", 
            "30-inf"
        ]
        self.books_data = books_info

    def start(self):
        print("\nHello, and welcome to Librairie Bonheur D'Occasion Inc.! I'm your Bookbuddy bot, here to help you with personalized book recommendations.")
        self.buying_for()

    def buying_for(self):
        response = input("\nTo get started, please let me know: are you buying for yourself or for a friend? (Myself/Friend) ")
        if response.lower() == "myself":
            self.user_data["buying_for"] = "Myself"
            self.ask_reading_frequency()
        elif response.lower() == "friend":
            self.user_data["buying_for"] = "Friend"
            print("\nBooks make a perfect gift. I can recommend you some great best sellers.")
            self.ask_recipient_age_group()
        else:
            print("\nInvalid input. Please choose 'Myself' or 'Friend'.")
            self.buying_for()

    def ask_reading_frequency(self):
        print("\nHow often do you read books? (Default is Daily)")
        options = ["Daily", "Weekly", "Monthly", "Occasionally"]
        for i, option in enumerate(options, 1):
            print(f"{i}- {option}")
        
        response = input("\nPlease choose a number or type your reading frequency: ")
        if response.isdigit() and 1 <= int(response) <= len(options):
            self.user_data["reading_frequency"] = options[int(response) - 1]
            if self.user_data["reading_frequency"] == "Occasionally":
                self.recommend_new_bestsellers()
            else:
                self.user_preference()
        elif any(response.lower() in option.lower() for option in options):
            matching_option = [option for option in options if response.lower() in option.lower()][0]
            self.user_data["reading_frequency"] = matching_option
            if self.user_data["reading_frequency"] == "Occasionally":
                self.recommend_new_bestsellers()
            else:
                self.user_preference()
        else:
            print("\nInvalid input. Please try again.")
            self.ask_reading_frequency()

    def recommend_new_bestsellers(self):
        print("\nHere are some of the new bestsellers:")
        recommended_books = [book for book, details in self.books_data.items() if details["bestseller"] == 1]
        
        if recommended_books:
            for i, book in enumerate(recommended_books[:10], 1):  # Show only top 10 recommendations
                author_name = self.books_data[book]["author"]
                print(f"{i}. {book} by {author_name}")
            self.proceed_with_recommendation(is_initial_recommendation=True)
        else:
            print("\nNo matching books found.")
            self.user_preference()

    def ask_recipient_age_group(self):
        print("\nAre you looking for a book for adults, young adults, or children?")
        options = ["Adults", "Young Adults", "Children"]
        for i, option in enumerate(options, 1):
            print(f"{i}- {option}")
        
        response = input("\nPlease choose a number or type the age group: (Default is Adults)")
        if response.isdigit() and 1 <= int(response) <= len(options):
            self.user_data["recipient_age_group"] = options[int(response) - 1]
            self.recommend_books_for_friend()
        elif any(response.lower() in option.lower() for option in options):
            matching_option = [option for option in options if response.lower() in option.lower()][0]
            self.user_data["recipient_age_group"] = matching_option
            self.recommend_books_for_friend()
        else:
            print("\nInvalid input. Please try again.")
            self.ask_recipient_age_group()

    def user_preference(self):
        if self.user_data["buying_for"].lower() == "friend":
            response = input("\nDoes your friend enjoy reading Fiction or Non-Fiction? (1- Fiction, 2- Non-Fiction) ")
        else:
            response = input("\nDo you usually enjoy reading Fiction or Non-Fiction? (1- Fiction, 2- Non-Fiction) ")
            
        if response == "1":
            self.user_data["preference"] = "Fiction"
            self.display_categories("fiction")
        elif response == "2":
            self.user_data["preference"] = "Non-Fiction"
            self.display_categories("non-fiction")
        else:
            print("\nInvalid input. Please choose '1' or '2'.")
            self.user_preference()

    def display_categories(self, category_type):
        print("\nGreat choice! Here are some popular categories: (Press enter to see all categories)")
        categories = self.categories[category_type][:10]
        for i, category in enumerate(categories, 1):
            print(f"{i}- {category}")

        if self.user_data["buying_for"].lower() == "friend":
            response = input("\nPlease choose a number or type your friend's preferred category: ")
        else:
            response = input("\nPlease choose a number or type your preferred category: ")
                    
        if response.isdigit() and 1 <= int(response) <= len(categories):
            self.user_data["subcategory"] = categories[int(response) - 1]
            self.get_author()
        else:
            self.check_custom_category(response, category_type)

    def check_custom_category(self, response, category_type):
        matching_categories = [cat for cat in self.categories[category_type] if response.lower() in cat.lower()]
        if matching_categories:
            print("\nDid you mean one of these categories?")
            for i, category in enumerate(matching_categories, 1):
                print(f"{i}- {category}")
            
            response = input("\nPlease choose a number from the above options or type 'no' to try again: ")
            if response.isdigit() and 1 <= int(response) <= len(matching_categories):
                self.user_data["subcategory"] = matching_categories[int(response) - 1]
                self.get_author()
            else:
                print("\nInvalid input or no matching categories selected. Please try again.")
                self.display_categories(category_type)
        else:
            print("\nNo matching categories found. Please try again.")
            self.display_categories(category_type)

    def get_author(self):
        if self.user_data["buying_for"].lower() == 'friend':
            author = input(f"\nWho is your friend's preferred author? ")
            self.user_data["author"] = author
            self.get_length()
        else:
            author = input(f"\nWho is your preferred author? ")
            self.user_data["author"] = author
            self.get_length()            

    def get_length(self):
        print("\nWhat is your preferred book length?")
        for i, length in enumerate(self.length_options, 1):
            print(f"{i}- {length}")
        
        response = input("\nPlease choose a number or type your preferred length: ")
        if response.isdigit() and 1 <= int(response) <= len(self.length_options):
            self.user_data["length"] = self.length_options[int(response) - 1].split(" (")[0]  # Removing the page information
            self.get_price()
        elif any(response.lower() in length.lower() for length in self.length_options):
            matching_length = [length for length in self.length_options if response.lower() in length.lower()][0]
            self.user_data["length"] = matching_length.split(" (")[0]  # Removing the page information
            self.get_price()
        else:
            print("\nInvalid input. Please try again.")
            self.get_length()

    def get_price(self):
        print("\nWhat is your preferred price range?")
        for i, price in enumerate(self.price_options, 1):
            print(f"{i}- {price}")
        
        response = input("\nPlease choose a number or type your preferred price range: ")
        if response.isdigit() and 1 <= int(response) <= len(self.price_options):
            min_price, max_price = map(float, self.price_options[int(response) - 1].split('-'))
            self.user_data["price"] = (min_price + max_price) / 2
            self.recommend_books()
        else:
            try:
                min_price, max_price = map(float, response.split('-'))
                self.user_data["price"] = (min_price + max_price) / 2
                self.recommend_books()
            except ValueError:
                print("\nInvalid input. Please try again.")
                self.get_price()

    def recommend_books(self):
        if self.user_data["buying_for"].lower() == "friend":
            self.recommend_books_for_friend()
        else:
            self.recommend_books_for_self()

    def recommend_books_for_friend(self):
        age_group = self.user_data.get("recipient_age_group", "")
        print(f"\nBased on your preference for bestsellers for {age_group}, here are some recommendations:")

        recommended_books = [book for book, details in self.books_data.items() if details["bestseller"] == 1 and details["age_group"].lower() == age_group.lower()]
        
        if recommended_books:
            for i, book in enumerate(recommended_books[:10], 1):  # Show only top 10 recommendations
                author_name = self.books_data[book]["author"]
                print(f"{i}. {book} by {author_name}")
            self.proceed_with_recommendation(is_initial_recommendation=False)
        else:
            print("\nNo matching books found.")
            self.ask_for_additional_preferences()

    def recommend_books_for_self(self):
        author = self.user_data.get("author", "")
        subcategory = self.user_data.get("subcategory", "")
        length = self.user_data.get("length", "")
        price = self.user_data.get("price", 0)
        
        print(f"\nBased on your preference for {subcategory} books by {author}, length {length}, and price around {price}, here are some recommendations:")
        recommended_books = [book for book, details in self.books_data.items() if
                     (author == "" or details["author"].lower() == author.lower() or 
                      (subcategory == "" or subcategory in details["genre"])) and 
                     details["length"].lower() == length.lower() and 
                     abs(details["price"] - price) <= 5]
        if recommended_books:
            for i, book in enumerate(recommended_books[:10], 1):  # Show only top 10 recommendations
                author_name = self.books_data[book]["author"]
                print(f"{i}. {book} by {author_name}")
        else:
            print("\nNo matching books found.")
        
        self.proceed_with_recommendation(is_initial_recommendation=False)

    def proceed_with_recommendation(self, is_initial_recommendation):
        response = input("\nWould you like to proceed with any of these recommendations? (Yes/No) ")
        if response.lower() == "yes":
            self.get_book_choice()
        elif response.lower() == "no":
            if is_initial_recommendation:
                self.ask_for_additional_preferences()
            else:
                try:
                    self.user_data["personal_choice_for_friend"]
                    print("\nThank you for using our service! Hope you enjoyed.")
#                     self.display_user_data()
                except KeyError:
                    if self.user_data["buying_for"].lower() == "friend":
                        self.user_data["personal_choice_for_friend"] = 1
                        self.user_preference()
                    else:
                        
                        print("\nThank you for using our service! Hope you enjoyed.")
#                         self.display_user_data()
        else:
            print("\nInvalid input. Please choose 'Yes' or 'No'.")
            self.proceed_with_recommendation(is_initial_recommendation)

    def ask_for_additional_preferences(self):
        print("\nLet's refine your search. Please provide additional preferences.")
        self.user_preference()

    def get_book_choice(self):
        book_choice = input("\nSure! Which one? ")
        if book_choice.isdigit() and 1 <= int(book_choice) <= 10:
            chosen_book_index = int(book_choice) - 1
            chosen_book = list(self.books_data.keys())[chosen_book_index]
            self.user_data["chosen_book"] = chosen_book
            self.get_email_address()
        else:
            print("\nInvalid input. Please provide a valid index number.")
            self.get_book_choice()

    def get_email_address(self):
        email = input("\nWhat is your email address? ")
        self.user_data["email"] = email
        print(f"\nWe have received your order for {self.user_data['chosen_book']} and will contact you once it is shipped!")
        print("\nThank you for using our service! Hope you enjoyed.")
#         self.display_user_data()

    def display_user_data(self):
        print("\nHere's a summary of your data:")
        for key, value in self.user_data.items():
            print(f"{key}: {value}")

if __name__ == "__main__":
    bot = BookBuddyBot()
    bot.start()